In [ ]:
import re

def remove_unwanted_symbols(sent: str):
    """

    remove unwanted symbols

    """
    
    
    symbols = '[\(\)|/#&%*\[\]《》\{\}~<=>`——-]+'
    sent = re.sub(symbols, "", sent)
    return sent.strip()

In [ ]:
print(remove_unwanted_symbols("Hel{l}o..."))

In [6]:
!pip install camel-tools
!camel_data full

In [9]:
import camel_tools as camel
from camel_tools.utils.stringutils import force_encoding, force_unicode
from camel_tools.utils.charmap import CharMapper

def ar_clean(mapper, sent: str):
    sent = force_unicode(sent)
    return mapper.map_string(sent)


mapper = CharMapper.builtin_mapper('arclean')
print(ar_clean(mapper, "تُريدُ إخْباري ماذا أنت تَعتقدُ 122؟"))

تريد إخباري ماذا أنت تعتقد 122?


In [10]:
from camel_tools.tokenizers.word import simple_word_tokenize
print(simple_word_tokenize("اريد ان اذهب الى السينما, المتحف"))

['اريد', 'ان', 'اذهب', 'الى', 'السينما', ',', 'المتحف']


In [11]:
# Go to URL
def rmURL(line):
    import  re
    pattern = re.compile(r'http[a-zA-Z0-9.?/&=:]*')
    return pattern.sub('', line.strip())
print(rmURL("mohamed is going to visit http://mabdelrehim.com/about jhjh"))

mohamed is going to visit  jhjh


In [12]:
!pip install demoji

In [13]:
import demoji
demoji.download_codes()
print(demoji.replace_with_desc("'I bet you didn't know that 🙋, 🙋‍♂️, and 🙋‍♀️ are three different emojis."))

... OK (Got response in 0.16 seconds)
Writing emoji data to /home/nlp-mt/.demoji/codes.json ...
... OK
'I bet you didn't know that :person raising hand:, :person raising hand:‍♂️, and :woman raising hand: are three different emojis.


In [14]:
import string
print(string.printable)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



In [15]:
def lang_detect(src_sent: str, trg_sent: str, src_lang: str, trg_lang: str , fasttext_model):

    """

    Takes a parallel pair of sentences in a parallel corpus and returns whether to remove the pair
    or not based on
        1- A large portion of the source sentence is in a language other than the source language
        2- A large portion of the target sentence is in a language other than the target language

    """

    detected_src, confidence_src = fasttext_model.predict(src_sent, k=1)
    detected_src = detected_src[0].replace("__label__", "")
    confidence_src = confidence_src[0]

    detected_trg, confidence_trg = fasttext_model.predict(trg_sent, k=1)
    detected_trg = detected_trg[0].replace("__label__", "")
    confidence_trg = confidence_trg[0]

    #print(detected_src,"|||||||",detected_src)

    if (not (detected_src == src_lang)) or (not (detected_trg == trg_lang)):
        print(f'src {detected_src} trg {detected_trg}')
        return True
    elif (confidence_src < 0.85) or (confidence_trg < 0.85):
        print(f'confidence src {confidence_src} confidence trg {confidence_trg}')
        return True
    else:
        print(f'src {detected_src} trg {detected_trg}')
        print(f'confidence src {confidence_src} confidence trg {confidence_trg}')
        return False


In [16]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2021-02-19 13:09:34--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: 'lid.176.bin.1'

lid.176.bin.1       100%[===================>] 125.18M  13.5MB/s    in 9.5s    

2021-02-19 13:09:44 (13.2 MB/s) - 'lid.176.bin.1' saved [131266198/131266198]



In [18]:
import fasttext
fasttext_model = fasttext.load_model('lid.176.bin')
lang_detect(src_sent="诊所将继续遵循 PHE 制定的 COVID-19 方案，有感染风险者应转诊至控制通道，而不是亲自前往诊所就诊。", trg_sent="ستظل العيادات تتبع بروتوكول هيئة الصحة العامة في إنجلترا لكوفيد-19 فيما يتعلق بالأشخاص المعرضين لخطر الإصابة بالعدوى الذين يجب وضع إشارات تحذيرية لتحديدهم في مسار الاحتواء، عوضًا عن حضورهم جسديًّا.؟", src_lang="zh", trg_lang="ar" , fasttext_model=fasttext_model)

src zh trg ar
confidence src 0.9829135537147522 confidence trg 0.9992139935493469


False

In [19]:
!pip install laserembeddings
!pip install laserembeddings[zh]
!pip install laserembeddings[ja]
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [7]:
from laserembeddings import Laser
from sklearn.metrics.pairwise import cosine_similarity
laser = Laser()

def laser_filter(laser_model, src: str, trg: str, src_lang: str, trg_lang: str):
    """
    
    The laser model tries to embed sentences that are of the same meaning with similar embeddings
    basic experimentation tells that sentences with cosine similarity score >= 0.8 are of acceptable quality
    
    """
    embeddings_a = laser_model.embed_sentences(
        [src],
        lang=src_lang)
    embeddings_b = laser_model.embed_sentences(
        [trg],
        lang=trg_lang)
    similarity_score = cosine_similarity(embeddings_a, embeddings_b).flatten()[0]
    return similarity_score
    

In [12]:
laser_filter(laser, "The ball is in the field", "في ماتش كورة بكرة مش عايزين نفوته", "en", "ar")

0.521637

In [27]:

import argparse
from pathlib import Path
import pandas as pd
import os
import time
import fasttext
from laserembeddings import Laser
from tqdm import tqdm
import numpy as np



def batched_laser_filter(laser_model, sources, targets, src_lang: str, trg_lang: str, batch_size=4096):
    """
    
    The laser model tries to embed sentences that are of the same meaning with similar embeddings
    basic experimentation tells that sentences with cosine similarity score >= 0.8 are of acceptable quality
    
    """
    start = time.time()
    i = 0
    embeddings_a = []
    with tqdm(total=len(sources) // batch_size) as pbar:
        while i < len(sources):
            embeddings_a.extend(laser_model.embed_sentences(
            sources[i:i+batch_size],
            lang=src_lang))
            i = i + batch_size
            pbar.update(1)
    end = time.time()
    elapsed = end - start
    print(f"---- Calculated laser embeddings for {len(sources)} sentences from source language in {elapsed // 60},  mins , {elapsed % 60} ,  secs")
    print(np.array(embeddings_a).shape)
    start = time.time()
    i = 0
    embeddings_b = []
    with tqdm(total=len(targets) // batch_size) as pbar:
        while i < len(targets):
            embeddings_b.extend(laser_model.embed_sentences(
            targets[i:i+batch_size],
            lang=trg_lang))
            i = i + batch_size
            pbar.update(1)
    end = time.time()
    elapsed = end - start
    print(f"---- Calculated laser embeddings for {len(targets)} sentences from target language in {elapsed // 60},  mins , {elapsed % 60},  secs")
    print(np.array(embeddings_b).shape)
    similarities = []
    start = time.time()
    for a, b in tqdm(zip(embeddings_a, embeddings_b)):
        similarities.append(cosine_similarity([a], [b]).flatten()[0])
    end = time.time()
    elapsed = end - start
    print(f"---- Calculated cosine for {len(targets)} pairs in {elapsed // 60},  mins , {elapsed} % 60,  secs")
    return similarities


laser = Laser()
data_file_src = open(os.path.join("../tico-19", "data" + "." + "zh" + ".clean"), "r")
data_file_trg = open(os.path.join("../tico-19", "data" + "." + "en" + ".clean"), "r")

src_sents = data_file_src.readlines()
trg_sents = data_file_trg.readlines()

df = pd.DataFrame(
    {
        "zh": src_sents,
        "en": trg_sents,
        "similarities": batched_laser_filter(laser, src_sents, trg_sents, "zh", "en")
    }
)
# apply length alignment filter
before = len(df)
start = time.time()
df = df[df['similarities'] < 0.7]
df = df[[args.src_lang, args.trg_lang]]
df = df.reset_index()
end = time.time()
after = len(df)
removed = before - after
elapsed = end - start
print("---- Removed ", removed, " pairs that are too different in lengths", elapsed // 60, " mins ", elapsed % 60, " secs")


1it [00:07,  7.93s/it]
0it [00:00, ?it/s]

---- Calculated laser embeddings for 3035 sentences from source language in 0.0,  mins , 7.928884983062744 ,  secs
(3035, 1024)


1it [00:02,  2.44s/it]
344it [00:00, 3431.58it/s]

---- Calculated laser embeddings for 3035 sentences from target language in 0.0,  mins , 2.4450862407684326,  secs
(3035, 1024)


3035it [00:00, 3301.69it/s]

---- Calculated cosine for 3035 pairs in 0.0,  mins , 0.9201710224151611 % 60,  secs


In [20]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([[1, 1, 1]], [[1, 10, 100]]))

[[0.63764677]]


In [26]:
df.head(50)

,zh,en,similarities
0,这些 症状 已 持续 多长时间 ？\n,about how long have these symptoms been going ...,[[0.8719396]]
1,各种 胸痛 均 应 采取 这种 方法 进行 治疗 ， 尤其 要 考虑 年龄 因素\n,and all chest pain should be treated this way ...,[[0.7698772]]
2,还 需要 检查一下 胆固醇 和 血压\n,and also needs to be checked your cholesterol ...,[[0.8304562]]
3,您 现在 有 发热 吗 ？\n,and are you having a fever now ?\n,[[0.76542723]]
4,您 的 胸痛 伴有 以下 任何 症状 吗\n,and are you having any of the following sympto...,[[0.7107259]]
5,您 有 流鼻涕 吗 ？\n,and are you having a runny nose ?\n,[[0.7137053]]
6,现在 您 有 这种 胸痛 症状 吗 ？\n,and are you having this chest pain now ?\n,[[0.80969715]]
7,另外 您 有 呼吸困难 吗\n,and besides do you have difficulty breathing\n,[[0.70627165]]
8,您 能 描述 一下 除此之外 还有 什么 其他 症状 吗 ？\n,and can you tell me what other symptoms are yo...,[[0.8041169]]
9,这种 疼痛 是从 您 胸部 延伸 出来 的 吗 ？\n,and does this pain move from your chest ?\n,[[0.79879177]]
